In [1]:
import time
import os
import obspy
from obspy import UTCDateTime
from obspy.core.event import  Event, Origin, Magnitude, Pick, WaveformStreamID
from obspy import Catalog
import pandas as pd
import pylab as plt
import numpy as np
from obspy import read_inventory
from obspy.clients.fdsn import Client
from obspy.clients.filesystem.sds import Client as sdsclient
from obspy.clients.fdsn import RoutingClient
from obspy.core import Trace, Stream, Stats
import glob
from obspy import UTCDateTime, Catalog


In [2]:

client = Client("INGV")
sds=sdsclient("/home/jovyan/data/sds/")
sdsYR=sdsclient("/home/jovyan/data/iris/")



In [5]:
starttime=UTCDateTime("2016-10-20T00:00:00")
endtime=UTCDateTime("2016-12-01T00:00:00")
print(starttime,endtime)

nday=int((endtime-starttime)/86400)
print(nday)

2016-10-20T00:00:00.000000Z 2016-12-01T00:00:00.000000Z
42


In [14]:

starttime = UTCDateTime("2016-10-20T00:00:00")
endtime   = UTCDateTime("2016-12-01T00:00:00")
nday = int((endtime - starttime) / 86400)

lon_min, lon_max = 12.9, 13.3
lat_min, lat_max = 42.7, 42.95

catINGV = []

for i in range(nday):
    day_start = starttime + i * 86400
    day_end   = day_start + 86400

    print(f"\n--- Giorno {i+1}/{nday}: {day_start} - {day_end} ---")

    catalog = client.get_events(
        starttime=day_start,
        endtime=day_end,
        minlatitude=lat_min,
        maxlatitude=lat_max,
        minlongitude=lon_min,
        maxlongitude=lon_max,
        includearrivals=False
    )

    print(f"  Trovati {len(catalog)} eventi")

    for ev in catalog:
        eid = str(ev.resource_id).split("eventId=")[-1]
        print(f"   -> Scarico dettagli per EventID: {eid}")

        ev_full = client.get_events(
            eventid=eid,
            includearrivals=True
        )[0]

        catINGV.append(ev_full)


# Converto la lista in un unico Catalog ObsPy
catalog_xml = Catalog(catINGV)

# Salvo in formato QuakeML
catalog_xml.write("catalog_ingv.xml", format="QUAKEML")

print("\n✅ Catalogo unico salvato come catalog_ingv.xml")



--- Giorno 1/42: 2016-10-20T00:00:00.000000Z - 2016-10-21T00:00:00.000000Z ---
  Trovati 120 eventi
   -> Scarico dettagli per EventID: 8598911
   -> Scarico dettagli per EventID: 8598851
   -> Scarico dettagli per EventID: 8598771
   -> Scarico dettagli per EventID: 8598591
   -> Scarico dettagli per EventID: 8598541
   -> Scarico dettagli per EventID: 8598461
   -> Scarico dettagli per EventID: 8598451
   -> Scarico dettagli per EventID: 8598371
   -> Scarico dettagli per EventID: 8598331
   -> Scarico dettagli per EventID: 8598271
   -> Scarico dettagli per EventID: 8598261
   -> Scarico dettagli per EventID: 8598241
   -> Scarico dettagli per EventID: 8598221
   -> Scarico dettagli per EventID: 8598201
   -> Scarico dettagli per EventID: 8598151
   -> Scarico dettagli per EventID: 8598011
   -> Scarico dettagli per EventID: 8597911
   -> Scarico dettagli per EventID: 8597781
   -> Scarico dettagli per EventID: 8597731
   -> Scarico dettagli per EventID: 8597691
   -> Scarico detta

In [ ]:
for i in range(nday):
    t0=starttime+i*86400
    t1=t0+86400
    print(t0,t1)

    inv_ingv_hh=client.get_stations(network="IV,XO,MN,8P",channel="HH*",starttime=t0,endtime=t1,latitude=center[1],longitude=center[0],maxradius=0.8,level="response")
    inv_ingv_eh=client.get_stations(network="IV,XO,MN,8P",channel="EH*",starttime=t0,endtime=t1,latitude=center[1],longitude=center[0],maxradius=0.8,level="response")
    inv_ingv_hn=client.get_stations(network="IV,XO,MN,8P",channel="HN*",starttime=t0,endtime=t1,latitude=center[1],longitude=center[0],maxradius=0.8,level="response")

    for net in inv_ingv_eh:
        for sta in net:
            trovo=inv_ingv_hh.select(station=sta.code)
            if trovo.select(station=sta.code).networks == []:
                inv_ingv_hh.extend(inv_ingv_eh.select(station=sta.code))
    for net in inv_ingv_hn:
        for sta in net:
            trovo=inv_ingv_hh.select(station=sta.code)
            if trovo.select(station=sta.code).networks == []:
                inv_ingv_hh.extend(inv_ingv_hn.select(station=sta.code))
    inv_ingv_hh.write("./INVENTORY/inventory_ingv"+str(t0.julday)+".xml",format="STATIONXML")